# Markov Decision Processes (MDPs)

Welcome back! In the previous notebook, we learned what reinforcement learning is through fun analogies. Now, let's learn the **mathematical language** that lets us describe RL problems precisely.

Don't worry - we'll still use lots of analogies and visualizations!

## What You'll Learn

By the end of this notebook, you'll understand:
- What an MDP is (with a board game analogy!)
- The five components: States, Actions, Transitions, Rewards, Discount
- The Markov property (why "history doesn't matter")
- How to model any decision problem as an MDP
- You'll build and visualize your own MDP!

**Prerequisites:** Notebook 1 (What is RL?)

**Time:** ~30 minutes

---
## The Big Picture: A Board Game Analogy

Imagine you're playing a board game like **Snakes and Ladders**:

```
    ┌────────────────────────────────────────────────┐
    │  🎲 SNAKES AND LADDERS 🎲                      │
    ├────────────────────────────────────────────────┤
    │                                                │
    │  STATES: Which square you're on (1-100)        │
    │                                                │
    │  ACTIONS: Roll the dice                        │
    │                                                │
    │  TRANSITIONS: Where you land                   │
    │    - Normal squares: move forward              │
    │    - Ladders: jump up! 🪜                       │
    │    - Snakes: slide down! 🐍                    │
    │                                                │
    │  REWARDS: +100 for reaching square 100!        │
    │                                                │
    └────────────────────────────────────────────────┘
```

This simple game has all the components of an **MDP**!

An MDP (Markov Decision Process) is just a formal way to describe:
- **Where you are** (states)
- **What you can do** (actions)
- **What happens** when you act (transitions)
- **How good/bad** the outcome is (rewards)
- **How much you care** about the future (discount)

---
## The Five Components of an MDP

Every MDP has exactly **five components**. Let's learn them with a simple example: **a robot navigating a warehouse**.

```
    ┌─────────────────────────────────────────────────────────────────┐
    │                      WAREHOUSE ROBOT MDP                        │
    ├─────────────────────────────────────────────────────────────────┤
    │                                                                 │
    │  S (States)       = Robot's location in the warehouse          │
    │                     {(0,0), (0,1), (1,0), (1,1), ...}          │
    │                                                                 │
    │  A (Actions)      = Movement options                            │
    │                     {UP, DOWN, LEFT, RIGHT}                     │
    │                                                                 │
    │  P (Transitions)  = Where robot ends up after moving            │
    │                     P(new_location | current_location, action)  │
    │                                                                 │
    │  R (Rewards)      = Feedback for each action                    │
    │                     +10 deliver package, -1 per step            │
    │                                                                 │
    │  γ (Gamma)        = How much robot cares about future           │
    │                     0.99 = cares a lot about future             │
    │                                                                 │
    └─────────────────────────────────────────────────────────────────┘
```

Let's visualize this!

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, Circle, FancyArrowPatch
import numpy as np

# Create a beautiful visualization of MDP components
fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 11)
ax.axis('off')
ax.set_facecolor('#f8f9fa')

# Title
ax.text(7, 10.5, 'The 5 Components of an MDP', ha='center', 
        fontsize=22, fontweight='bold', color='#333')
ax.text(7, 9.8, 'MDP = (S, A, P, R, γ)', ha='center', 
        fontsize=16, color='#666', style='italic')

# Component boxes with descriptions
components = [
    {'name': 'S', 'full': 'States', 'color': '#e3f2fd', 'edge': '#1976d2',
     'desc': 'All possible situations', 'example': '"Where am I?"'},
    {'name': 'A', 'full': 'Actions', 'color': '#e8f5e9', 'edge': '#388e3c',
     'desc': 'What agent can do', 'example': '"What can I do?"'},
    {'name': 'P', 'full': 'Transitions', 'color': '#fff3e0', 'edge': '#f57c00',
     'desc': 'What happens next', 'example': '"Where will I end up?"'},
    {'name': 'R', 'full': 'Rewards', 'color': '#fce4ec', 'edge': '#c2185b',
     'desc': 'Feedback signal', 'example': '"Was that good or bad?"'},
    {'name': 'γ', 'full': 'Discount', 'color': '#f3e5f5', 'edge': '#7b1fa2',
     'desc': 'Future importance', 'example': '"How patient am I?"'},
]

# Draw component boxes in a row
start_x = 0.5
box_width = 2.4
box_height = 3
spacing = 0.35

for i, comp in enumerate(components):
    x = start_x + i * (box_width + spacing)
    y = 5.5
    
    # Draw box
    box = FancyBboxPatch((x, y), box_width, box_height, 
                          boxstyle="round,pad=0.1",
                          facecolor=comp['color'], 
                          edgecolor=comp['edge'], linewidth=3)
    ax.add_patch(box)
    
    # Symbol
    ax.text(x + box_width/2, y + 2.5, comp['name'], ha='center', va='center',
            fontsize=28, fontweight='bold', color=comp['edge'])
    
    # Full name
    ax.text(x + box_width/2, y + 1.8, comp['full'], ha='center', va='center',
            fontsize=12, fontweight='bold', color=comp['edge'])
    
    # Description
    ax.text(x + box_width/2, y + 1.2, comp['desc'], ha='center', va='center',
            fontsize=9, color='#555')
    
    # Example question
    ax.text(x + box_width/2, y + 0.5, comp['example'], ha='center', va='center',
            fontsize=9, color='#888', style='italic')

# Add a concrete example at the bottom
example_box = FancyBboxPatch((1, 1), 12, 3.5, boxstyle="round,pad=0.1",
                              facecolor='white', edgecolor='#333', linewidth=2)
ax.add_patch(example_box)

ax.text(7, 4, 'Example: Pac-Man Game', ha='center', fontsize=14, fontweight='bold', color='#333')
ax.text(2.5, 3.2, 'S: Pac-Man & ghost positions', fontsize=10, color='#1976d2')
ax.text(2.5, 2.6, 'A: UP, DOWN, LEFT, RIGHT', fontsize=10, color='#388e3c')
ax.text(7.5, 3.2, 'P: Move in chosen direction', fontsize=10, color='#f57c00')
ax.text(7.5, 2.6, 'R: +10 dot, +200 ghost, -500 die', fontsize=10, color='#c2185b')
ax.text(11, 2.9, 'γ = 0.99', fontsize=10, color='#7b1fa2')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("MEMORY TRICK: Think 'SAPRGAMMA'")
print("="*60)
print("\nS - States:      WHERE am I? (all possible situations)")
print("A - Actions:     WHAT can I do? (all possible choices)")
print("P - Probability: WHERE will I end up? (transition function)")
print("R - Rewards:     HOW GOOD was that? (feedback signal)")
print("γ - Gamma:       HOW PATIENT am I? (discount factor)")
print("\n" + "="*60)

---
## The Markov Property: "History Doesn't Matter"

The most important assumption in MDPs is the **Markov property**.

### The GPS Analogy

Imagine you're using GPS navigation:

```
    ┌────────────────────────────────────────────────────────┐
    │                    GPS NAVIGATION                       │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Your GPS only needs to know:                          │
    │    ✓ Where you ARE (current location)                  │
    │    ✓ Where you want to GO (destination)                │
    │                                                         │
    │  Your GPS does NOT need to know:                       │
    │    ✗ Where you started from                            │
    │    ✗ What route you took to get here                   │
    │    ✗ How many wrong turns you made                     │
    │                                                         │
    │  The current location has ALL the information          │
    │  needed to find the best route forward!                │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

This is exactly what the **Markov property** says:

> **The future depends only on where you ARE, not how you GOT there.**

### Chess Example

In chess, the optimal move depends only on the current board position:
- It doesn't matter if you opened with the Sicilian Defense or Queen's Gambit
- It doesn't matter if you made mistakes earlier
- All that matters is: where are the pieces RIGHT NOW?

In [ ]:
# Visualize the Markov Property
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: NON-Markov (history matters)
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 6)
ax1.axis('off')
ax1.set_title('WITHOUT Markov Property\n(History matters - complicated!)', 
              fontsize=14, color='#d32f2f', fontweight='bold')

# Draw states with history
states_left = [(2, 4.5, 'State A'), (5, 4.5, 'State B'), (8, 4.5, 'State C')]
for x, y, label in states_left:
    circle = Circle((x, y), 0.6, facecolor='#ffcdd2', edgecolor='#d32f2f', linewidth=2)
    ax1.add_patch(circle)
    ax1.text(x, y, label.split()[1], ha='center', va='center', fontsize=12)

# Arrows with history labels
ax1.annotate('', xy=(4.3, 4.5), xytext=(2.7, 4.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f'))
ax1.annotate('', xy=(7.3, 4.5), xytext=(5.7, 4.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f'))

# Decision at C depends on path
ax1.text(5, 2.5, 'To decide at C, need to know:', fontsize=11, ha='center')
ax1.text(5, 2.0, '• Did I come from A→B→C?', fontsize=10, ha='center', color='#666')
ax1.text(5, 1.5, '• Or from D→B→C?', fontsize=10, ha='center', color='#666')
ax1.text(5, 1.0, '• Or from A→E→B→C?', fontsize=10, ha='center', color='#666')
ax1.text(5, 0.4, '😰 Must remember ENTIRE history!', fontsize=10, ha='center', color='#d32f2f')

# Right: Markov (only current state matters)
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 6)
ax2.axis('off')
ax2.set_title('WITH Markov Property\n(Only current state matters - simple!)', 
              fontsize=14, color='#388e3c', fontweight='bold')

# Draw current state prominently
circle = Circle((5, 4), 1.2, facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax2.add_patch(circle)
ax2.text(5, 4, 'C', ha='center', va='center', fontsize=24, fontweight='bold')
ax2.text(5, 2.5, 'Current\nState', ha='center', fontsize=12, color='#388e3c')

# Faded past
for x, y, label, alpha in [(2, 5, 'A', 0.3), (2, 3, 'D', 0.3), (3.5, 4.5, 'B', 0.3)]:
    circle = Circle((x, y), 0.5, facecolor='#e0e0e0', edgecolor='#9e9e9e', 
                     linewidth=1, alpha=alpha)
    ax2.add_patch(circle)
    ax2.text(x, y, label, ha='center', va='center', fontsize=10, alpha=alpha)

ax2.text(2.5, 1.5, '(Past doesn\'t matter)', fontsize=10, ha='center', color='#9e9e9e')

# Decision only depends on C
ax2.text(7.5, 4, 'Decision only\ndepends on C!', fontsize=12, ha='center', color='#388e3c')
ax2.text(7.5, 2.5, '😊 Simple!', fontsize=14, ha='center', color='#388e3c')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("THE MARKOV PROPERTY")
print("="*60)
print("\n'The future is independent of the past, given the present.'")
print("\nIn math: P(future | present, past) = P(future | present)")
print("\nIn plain English: If I know where I AM, I don't need to know")
print("                  how I GOT here to make the best decision!")
print("\n" + "="*60)

---
## Let's Build a Grid World MDP!

Now let's create a real MDP and explore its components. We'll build the same grid world from Notebook 1, but this time we'll look at it through the lens of an MDP.

```
    ┌───┬───┬───┬───┐
    │ R │   │   │   │   R = Robot starts here (0,0)
    ├───┼───┼───┼───┤
    │   │   │   │   │
    ├───┼───┼───┼───┤
    │   │   │   │   │
    ├───┼───┼───┼───┤
    │   │   │   │ G │   G = Goal (3,3)
    └───┴───┴───┴───┘
    
    S = {(0,0), (0,1), ..., (3,3)}  (16 states)
    A = {UP, RIGHT, DOWN, LEFT}     (4 actions)
    P = Deterministic movement       (action always succeeds)
    R = -1 per step, +10 at goal
    γ = 0.99
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

class GridWorldMDP:
    """
    A 4x4 Grid World as an MDP.
    
    This is a complete MDP with all 5 components:
    - S: 16 states (grid positions)
    - A: 4 actions (UP, RIGHT, DOWN, LEFT)
    - P: Deterministic transitions
    - R: -1 per step, +10 at goal
    - γ: 0.99 (cares a lot about future)
    """
    
    def __init__(self):
        # ========================================
        # Component 1: STATE SPACE (S)
        # ========================================
        self.grid_size = 4
        self.n_states = self.grid_size ** 2  # 16 states
        self.states = [(r, c) for r in range(self.grid_size) 
                              for c in range(self.grid_size)]
        
        # ========================================
        # Component 2: ACTION SPACE (A)
        # ========================================
        self.n_actions = 4
        self.actions = [0, 1, 2, 3]  # UP, RIGHT, DOWN, LEFT
        self.action_names = ['UP', 'RIGHT', 'DOWN', 'LEFT']
        self.action_symbols = ['↑', '→', '↓', '←']
        
        # ========================================
        # Component 5: DISCOUNT FACTOR (γ)
        # ========================================
        self.gamma = 0.99  # Agent cares about future rewards
        
        # Special states
        self.goal = (3, 3)  # Bottom-right corner
        
        # Build transition and reward functions
        self._build_dynamics()
    
    def _build_dynamics(self):
        """
        Build Components 3 and 4: Transitions (P) and Rewards (R)
        """
        # ========================================
        # Component 3: TRANSITION FUNCTION (P)
        # P[state][action] = [(probability, next_state), ...]
        # ========================================
        self.P = {}
        
        # ========================================
        # Component 4: REWARD FUNCTION (R)
        # R[state][action] = reward
        # ========================================
        self.R = {}
        
        for state in self.states:
            self.P[state] = {}
            self.R[state] = {}
            
            for action in self.actions:
                next_state = self._get_next_state(state, action)
                reward = self._get_reward(state, action, next_state)
                
                # Deterministic: probability 1.0 for the next state
                self.P[state][action] = [(1.0, next_state)]
                self.R[state][action] = reward
    
    def _get_next_state(self, state, action):
        """Determine next state given current state and action."""
        row, col = state
        
        if action == 0:    # UP
            row = max(0, row - 1)
        elif action == 1:  # RIGHT
            col = min(self.grid_size - 1, col + 1)
        elif action == 2:  # DOWN
            row = min(self.grid_size - 1, row + 1)
        elif action == 3:  # LEFT
            col = max(0, col - 1)
        
        return (row, col)
    
    def _get_reward(self, state, action, next_state):
        """Determine reward for a transition."""
        if next_state == self.goal:
            return 10   # Big reward for reaching goal!
        return -1       # Small penalty for each step
    
    def visualize_mdp(self):
        """Create a visual representation of the MDP."""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 1. State Space
        ax1 = axes[0]
        ax1.set_title('STATE SPACE (S)\n16 grid positions', fontsize=14, fontweight='bold')
        
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                color = '#c8e6c9' if (r, c) == self.goal else '#e3f2fd'
                rect = plt.Rectangle((c, 3-r), 1, 1, facecolor=color, 
                                       edgecolor='black', linewidth=2)
                ax1.add_patch(rect)
                ax1.text(c+0.5, 3-r+0.5, f'({r},{c})', ha='center', va='center', fontsize=10)
        
        ax1.text(3.5, 0.5, 'GOAL', ha='center', va='center', fontsize=9, fontweight='bold', color='green')
        ax1.set_xlim(0, 4)
        ax1.set_ylim(0, 4)
        ax1.set_aspect('equal')
        ax1.axis('off')
        
        # 2. Action Space
        ax2 = axes[1]
        ax2.set_title('ACTION SPACE (A)\n4 movement directions', fontsize=14, fontweight='bold')
        ax2.set_xlim(0, 4)
        ax2.set_ylim(0, 4)
        ax2.axis('off')
        
        # Draw action arrows from center
        center = (2, 2)
        circle = plt.Circle(center, 0.3, facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
        ax2.add_patch(circle)
        ax2.text(center[0], center[1], 'S', ha='center', va='center', fontsize=12, fontweight='bold')
        
        arrows = [
            ((2, 2.4), (2, 3.2), '↑ UP', '#4caf50'),
            ((2.4, 2), (3.2, 2), '→ RIGHT', '#2196f3'),
            ((2, 1.6), (2, 0.8), '↓ DOWN', '#ff9800'),
            ((1.6, 2), (0.8, 2), '← LEFT', '#9c27b0')
        ]
        
        for (x1, y1), (x2, y2), label, color in arrows:
            ax2.annotate('', xy=(x2, y2), xytext=(x1, y1),
                        arrowprops=dict(arrowstyle='->', lw=3, color=color))
        
        ax2.text(2, 3.6, 'UP (0)', ha='center', fontsize=10, color='#4caf50')
        ax2.text(3.6, 2, 'RIGHT (1)', ha='center', fontsize=10, color='#2196f3')
        ax2.text(2, 0.4, 'DOWN (2)', ha='center', fontsize=10, color='#ff9800')
        ax2.text(0.4, 2, 'LEFT (3)', ha='center', fontsize=10, color='#9c27b0')
        
        # 3. Rewards
        ax3 = axes[2]
        ax3.set_title('REWARD FUNCTION (R)\nFeedback for actions', fontsize=14, fontweight='bold')
        ax3.set_xlim(0, 4)
        ax3.set_ylim(0, 4)
        ax3.axis('off')
        
        # Draw reward structure
        ax3.text(2, 3.2, 'Each step: -1', fontsize=14, ha='center', color='#d32f2f',
                bbox=dict(boxstyle='round', facecolor='#ffebee', edgecolor='#d32f2f'))
        ax3.text(2, 2.2, '(Encourages efficiency!)', fontsize=10, ha='center', color='#666')
        
        ax3.text(2, 1.2, 'Reach goal: +10', fontsize=14, ha='center', color='#388e3c',
                bbox=dict(boxstyle='round', facecolor='#e8f5e9', edgecolor='#388e3c'))
        ax3.text(2, 0.5, f'Discount γ = {self.gamma}', fontsize=12, ha='center', color='#7b1fa2')
        
        plt.tight_layout()
        plt.show()

# Create and visualize our MDP
mdp = GridWorldMDP()

print("🎮 GRID WORLD MDP")
print("=" * 50)
print(f"\nS (States):      {mdp.n_states} grid positions")
print(f"A (Actions):     {mdp.n_actions} directions: {mdp.action_names}")
print(f"P (Transitions): Deterministic (actions always succeed)")
print(f"R (Rewards):     -1 per step, +10 at goal")
print(f"γ (Gamma):       {mdp.gamma} (cares about future)")
print("\n" + "=" * 50)

mdp.visualize_mdp()

---
## Exploring Transitions: What Happens When We Act?

Let's look at the transition function in detail. We'll pick a state and see what happens for each action.

In [ ]:
def visualize_transitions_from_state(mdp, state):
    """Visualize all transitions from a given state."""
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_xlim(-0.5, 4.5)
    ax.set_ylim(-0.5, 4.5)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Draw grid
    for r in range(mdp.grid_size):
        for c in range(mdp.grid_size):
            if (r, c) == mdp.goal:
                color = '#c8e6c9'
            elif (r, c) == state:
                color = '#bbdefb'
            else:
                color = 'white'
            rect = plt.Rectangle((c, 3-r), 1, 1, facecolor=color, 
                                   edgecolor='black', linewidth=2)
            ax.add_patch(rect)
    
    # Mark goal
    ax.text(3.5, 0.5, 'GOAL', ha='center', va='center', fontsize=10, 
            fontweight='bold', color='green')
    
    # Draw transitions from current state
    row, col = state
    center_x, center_y = col + 0.5, 3 - row + 0.5
    
    # Agent marker
    ax.text(center_x, center_y, '🤖', ha='center', va='center', fontsize=24)
    
    # Draw arrows for each action
    colors = ['#4caf50', '#2196f3', '#ff9800', '#9c27b0']
    offsets = [(-0.3, 0), (0, 0.3), (0.3, 0), (0, -0.3)]  # For labeling
    
    for action in mdp.actions:
        transitions = mdp.P[state][action]
        reward = mdp.R[state][action]
        
        for prob, next_state in transitions:
            next_row, next_col = next_state
            next_x, next_y = next_col + 0.5, 3 - next_row + 0.5
            
            # Draw arrow
            if next_state != state:  # Only draw if we actually move
                ax.annotate('', xy=(next_x, next_y), xytext=(center_x, center_y),
                           arrowprops=dict(arrowstyle='->', lw=3, color=colors[action]))
    
    # Title with transition info
    ax.set_title(f'Transitions from State {state}', fontsize=16, fontweight='bold')
    
    # Legend
    legend_y = -0.3
    for action in mdp.actions:
        next_state = mdp.P[state][action][0][1]
        reward = mdp.R[state][action]
        ax.text(action + 0.5, legend_y, 
                f"{mdp.action_names[action]}\n→ {next_state}\nR={reward:+d}", 
                ha='center', va='top', fontsize=9, color=colors[action])
    
    plt.tight_layout()
    plt.show()

# Visualize transitions from state (1, 1)
print("Let's see what happens when the agent is at position (1, 1):")
print("="*50)
visualize_transitions_from_state(mdp, (1, 1))

# Print detailed transition info
state = (1, 1)
print(f"\nDetailed transitions from state {state}:")
print("-" * 50)
for action in mdp.actions:
    transitions = mdp.P[state][action]
    reward = mdp.R[state][action]
    for prob, next_state in transitions:
        print(f"  {mdp.action_names[action]:6s} → {next_state}  "
              f"(prob={prob:.0%}, reward={reward:+d})")

---
## Stochastic Transitions: When Actions Don't Always Work

In the real world, actions don't always succeed. Imagine a robot on a **slippery floor**:

```
    ┌────────────────────────────────────────────────────────┐
    │                  SLIPPERY FLOOR MDP                    │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Robot tries to go RIGHT:                              │
    │                                                         │
    │    80% → Actually goes RIGHT (success!)                │
    │    10% → Slips and goes UP (oops!)                     │
    │    10% → Slips and goes DOWN (oops!)                   │
    │                                                         │
    │  This is a STOCHASTIC transition!                      │
    │  The outcome is partly random.                         │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

Let's create a slippery grid world!

In [ ]:
class SlipperyGridWorldMDP(GridWorldMDP):
    """
    A slippery grid world where actions succeed 80% of the time.
    With 10% probability each, the agent slips perpendicular.
    
    This demonstrates STOCHASTIC transitions!
    """
    
    def __init__(self, slip_prob=0.1):
        self.slip_prob = slip_prob
        self.success_prob = 1 - 2 * slip_prob
        super().__init__()
    
    def _build_dynamics(self):
        """Build STOCHASTIC transition function."""
        self.P = {}
        self.R = {}
        
        for state in self.states:
            self.P[state] = {}
            self.R[state] = {}
            
            for action in self.actions:
                transitions = []
                expected_reward = 0
                
                # Main action succeeds with probability (1 - 2*slip_prob)
                main_next = self._get_next_state(state, action)
                main_reward = self._get_reward(state, action, main_next)
                transitions.append((self.success_prob, main_next))
                expected_reward += self.success_prob * main_reward
                
                # Slip left (perpendicular)
                left_action = (action - 1) % 4
                left_next = self._get_next_state(state, left_action)
                left_reward = self._get_reward(state, left_action, left_next)
                transitions.append((self.slip_prob, left_next))
                expected_reward += self.slip_prob * left_reward
                
                # Slip right (perpendicular)
                right_action = (action + 1) % 4
                right_next = self._get_next_state(state, right_action)
                right_reward = self._get_reward(state, right_action, right_next)
                transitions.append((self.slip_prob, right_next))
                expected_reward += self.slip_prob * right_reward
                
                self.P[state][action] = transitions
                self.R[state][action] = expected_reward

# Create slippery MDP
slippery_mdp = SlipperyGridWorldMDP(slip_prob=0.1)

print("🧊 SLIPPERY GRID WORLD MDP")
print("="*50)
print(f"\nSlip probability: {slippery_mdp.slip_prob:.0%}")
print(f"Success probability: {slippery_mdp.success_prob:.0%}")

# Compare deterministic vs stochastic
state = (1, 1)
action = 1  # RIGHT

print(f"\n" + "="*50)
print(f"Comparison: Taking action RIGHT from state {state}")
print("="*50)

print(f"\n📦 DETERMINISTIC (normal grid):")
for prob, next_state in mdp.P[state][action]:
    print(f"   → {next_state} with {prob:.0%} probability")

print(f"\n🧊 STOCHASTIC (slippery grid):")
for prob, next_state in slippery_mdp.P[state][action]:
    print(f"   → {next_state} with {prob:.0%} probability")

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

def draw_grid(ax, mdp, state, title, is_stochastic=False):
    ax.set_xlim(-0.5, 4.5)
    ax.set_ylim(-0.5, 4.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # Draw grid
    for r in range(mdp.grid_size):
        for c in range(mdp.grid_size):
            color = 'white'
            if (r, c) == mdp.goal:
                color = '#c8e6c9'
            elif (r, c) == state:
                color = '#bbdefb'
            rect = plt.Rectangle((c, 3-r), 1, 1, facecolor=color, 
                                   edgecolor='black', linewidth=2)
            ax.add_patch(rect)
    
    row, col = state
    cx, cy = col + 0.5, 3 - row + 0.5
    ax.text(cx, cy, '🤖', ha='center', va='center', fontsize=20)
    
    # Draw transition for RIGHT action
    action = 1  # RIGHT
    for prob, next_state in mdp.P[state][action]:
        next_row, next_col = next_state
        nx, ny = next_col + 0.5, 3 - next_row + 0.5
        
        if next_state != state:
            # Arrow thickness based on probability
            lw = 2 + prob * 4
            alpha = 0.5 + prob * 0.5
            ax.annotate('', xy=(nx, ny), xytext=(cx, cy),
                       arrowprops=dict(arrowstyle='->', lw=lw, 
                                      color='#2196f3', alpha=alpha))
            ax.text(nx, ny - 0.3, f'{prob:.0%}', ha='center', fontsize=10, color='#2196f3')
    
    ax.text(2, 4.2, f'Action: RIGHT (→)', ha='center', fontsize=12, color='#2196f3')

# Deterministic
draw_grid(axes[0], mdp, (1, 1), '📦 Deterministic: 100% success')

# Stochastic
draw_grid(axes[1], slippery_mdp, (1, 1), '🧊 Stochastic: May slip!')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("KEY INSIGHT: Stochastic MDPs are harder to solve!")
print("="*60)
print("\nIn deterministic MDPs:")
print("  - You know exactly what will happen")
print("  - Planning is easier")
print("\nIn stochastic MDPs:")
print("  - Outcomes are uncertain")
print("  - Must plan for multiple possibilities")
print("  - More like the real world!")
print("\n" + "="*60)

---
## Modeling Real Problems as MDPs

Any sequential decision-making problem can be modeled as an MDP. Let's practice with some examples!

### The 5-Step MDP Design Process

```
    ┌─────────────────────────────────────────────────────────────┐
    │               5 STEPS TO CREATE AN MDP                      │
    ├─────────────────────────────────────────────────────────────┤
    │                                                             │
    │  1. STATES:  What information does the agent need?          │
    │              (What defines the "situation"?)                │
    │                                                             │
    │  2. ACTIONS: What can the agent do?                         │
    │              (What are the choices?)                        │
    │                                                             │
    │  3. TRANSITIONS: What happens after each action?            │
    │              (Is it deterministic or random?)               │
    │                                                             │
    │  4. REWARDS: What behavior do you want?                     │
    │              (How do you measure success?)                  │
    │                                                             │
    │  5. DISCOUNT: How patient should the agent be?              │
    │              (γ close to 1 = patient, close to 0 = greedy)  │
    │                                                             │
    └─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Let's design some real-world MDPs!

real_world_mdps = {
    "🎮 Video Game (Pac-Man)": {
        "States": "Position of Pac-Man, ghosts, remaining dots, power-up status",
        "Actions": "UP, DOWN, LEFT, RIGHT",
        "Transitions": "Mostly deterministic; ghosts move unpredictably",
        "Rewards": "+10 eat dot, +200 eat ghost, -500 die, +1000 win",
        "Discount": "0.99 (survive and score high)"
    },
    
    "🚗 Self-Driving Car": {
        "States": "Position, speed, camera images, sensor readings, traffic",
        "Actions": "Accelerate, brake, steer left/right, change lane",
        "Transitions": "Stochastic (traffic, weather, other drivers)",
        "Rewards": "+1/sec safe driving, +100 reach destination, -1000 crash",
        "Discount": "0.99 (care about reaching destination safely)"
    },
    
    "🤖 Robot Arm": {
        "States": "Joint angles, gripper state, object positions",
        "Actions": "Move each joint, open/close gripper",
        "Transitions": "Mostly deterministic (physics simulation)",
        "Rewards": "+10 pick up object, +100 place correctly, -1 time",
        "Discount": "0.95 (efficiency matters)"
    },
    
    "💹 Stock Trading": {
        "States": "Portfolio, cash, prices, market indicators",
        "Actions": "Buy/sell/hold each stock, amount",
        "Transitions": "Highly stochastic (market moves)",
        "Rewards": "Portfolio value change (daily P&L)",
        "Discount": "0.99 (long-term wealth building)"
    },
    
    "🩺 Medical Treatment": {
        "States": "Patient vitals, test results, treatment history",
        "Actions": "Prescribe medication, dosage, procedures",
        "Transitions": "Stochastic (patient response varies)",
        "Rewards": "+100 recovery, -10 side effects, -1000 adverse event",
        "Discount": "0.95 (patient health is priority)"
    }
}

print("="*70)
print("REAL-WORLD MDP EXAMPLES")
print("="*70)

for name, components in real_world_mdps.items():
    print(f"\n{name}")
    print("-" * 60)
    for comp, desc in components.items():
        print(f"  {comp:12s}: {desc}")

---
## Interactive: Design Your Own MDP!

Let's create a simple **thermostat MDP** together. A thermostat must decide when to turn heating on/off to maintain a comfortable temperature.

In [ ]:
class ThermostatMDP:
    """
    A simple thermostat MDP.
    
    The thermostat must keep the room at a comfortable temperature (20°C)
    while minimizing energy use.
    
    States: Temperature in the room (15°C to 25°C)
    Actions: HEAT_ON, HEAT_OFF
    Transitions: Temperature changes based on action and outside temp
    Rewards: Penalty for being far from 20°C, penalty for heating
    """
    
    def __init__(self):
        # State space: Temperatures from 15 to 25 (discretized)
        self.states = list(range(15, 26))  # 15, 16, ..., 25
        self.n_states = len(self.states)
        
        # Action space
        self.actions = [0, 1]  # 0 = HEAT_OFF, 1 = HEAT_ON
        self.action_names = ['HEAT_OFF', 'HEAT_ON']
        
        # Target temperature
        self.target_temp = 20
        self.outside_temp = 10  # Cold outside!
        
        # Discount factor
        self.gamma = 0.9
        
        # Build MDP
        self._build_dynamics()
    
    def _build_dynamics(self):
        self.P = {}
        self.R = {}
        
        for temp in self.states:
            self.P[temp] = {}
            self.R[temp] = {}
            
            for action in self.actions:
                # Temperature change based on action
                if action == 1:  # HEAT_ON
                    # Room heats up
                    change = 2 if temp < 25 else 0
                else:  # HEAT_OFF
                    # Room cools toward outside temp
                    change = -1 if temp > self.outside_temp else 0
                
                next_temp = max(15, min(25, temp + change))
                
                # Stochastic: 80% intended, 20% stay same
                self.P[temp][action] = [(0.8, next_temp), (0.2, temp)]
                
                # Reward: Negative of distance from target, minus heating cost
                comfort_penalty = -abs(next_temp - self.target_temp)
                heating_cost = -1 if action == 1 else 0
                self.R[temp][action] = comfort_penalty + heating_cost
    
    def visualize(self):
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Left: State-action transitions
        ax1 = axes[0]
        ax1.set_title('Thermostat State Space\n(Temperature)', fontsize=14, fontweight='bold')
        
        temps = self.states
        y_positions = range(len(temps))
        colors = ['#ffcdd2' if t < 18 else '#c8e6c9' if 19 <= t <= 21 else '#fff9c4' 
                  for t in temps]
        
        bars = ax1.barh(y_positions, [1]*len(temps), color=colors, edgecolor='black')
        ax1.set_yticks(y_positions)
        ax1.set_yticklabels([f'{t}°C' for t in temps])
        ax1.set_xlim(0, 1.5)
        ax1.set_xlabel('')
        ax1.axhline(5, color='green', linestyle='--', linewidth=2, label='Target: 20°C')
        
        # Add comfort zone annotation
        ax1.text(1.1, 5, '← Comfortable!', fontsize=10, color='green', va='center')
        ax1.text(1.1, 1, '← Too cold!', fontsize=10, color='red', va='center')
        ax1.text(1.1, 9, '← Too hot!', fontsize=10, color='orange', va='center')
        
        ax1.set_xticks([])
        
        # Right: Reward structure
        ax2 = axes[1]
        ax2.set_title('Reward Structure', fontsize=14, fontweight='bold')
        
        # Calculate rewards for each temperature
        heat_off_rewards = [self.R[t][0] for t in temps]
        heat_on_rewards = [self.R[t][1] for t in temps]
        
        x = np.arange(len(temps))
        width = 0.35
        
        ax2.bar(x - width/2, heat_off_rewards, width, label='HEAT_OFF', color='#90caf9')
        ax2.bar(x + width/2, heat_on_rewards, width, label='HEAT_ON', color='#ffab91')
        
        ax2.set_xlabel('Temperature')
        ax2.set_ylabel('Reward')
        ax2.set_xticks(x)
        ax2.set_xticklabels([f'{t}°' for t in temps])
        ax2.legend()
        ax2.axvline(5, color='green', linestyle='--', linewidth=2, alpha=0.5)
        
        plt.tight_layout()
        plt.show()

# Create and visualize
thermostat = ThermostatMDP()

print("🌡️ THERMOSTAT MDP")
print("="*50)
print(f"\nStates: Room temperature ({min(thermostat.states)}°C to {max(thermostat.states)}°C)")
print(f"Actions: {thermostat.action_names}")
print(f"Target: {thermostat.target_temp}°C")
print(f"Outside: {thermostat.outside_temp}°C (it's cold!)")
print(f"Discount: {thermostat.gamma}")
print("\n" + "="*50)

thermostat.visualize()

# Show example transitions
print("\nExample transitions from 18°C:")
for action in thermostat.actions:
    print(f"\n  {thermostat.action_names[action]}:")
    for prob, next_temp in thermostat.P[18][action]:
        reward = thermostat.R[18][action]
        print(f"    → {next_temp}°C with {prob:.0%} probability, reward = {reward}")

---
## Summary: Key Takeaways

Let's recap what we learned about MDPs!

### The 5 Components (S, A, P, R, γ)

| Component | Symbol | Question It Answers | Example (Pac-Man) |
|-----------|--------|---------------------|-------------------|
| **States** | S | Where am I? | Position on the maze |
| **Actions** | A | What can I do? | UP, DOWN, LEFT, RIGHT |
| **Transitions** | P | What happens? | Move in chosen direction |
| **Rewards** | R | Was it good? | +10 for eating a dot |
| **Discount** | γ | How patient? | 0.99 (care about future) |

### The Markov Property

- **"The future depends only on the present, not the past"**
- Like GPS: only needs current location, not your travel history
- Makes problems tractable!

### Deterministic vs Stochastic

- **Deterministic:** Actions always succeed (easier)
- **Stochastic:** Actions might fail (more realistic)

### Any Decision Problem → MDP

Follow the 5-step process to model any problem as an MDP!

---
## Test Your Understanding

**1. What are the 5 components of an MDP?**
<details>
<summary>Click to reveal answer</summary>
States (S), Actions (A), Transitions (P), Rewards (R), and Discount factor (γ).
</details>

**2. What does the Markov property mean?**
<details>
<summary>Click to reveal answer</summary>
The future depends only on the current state, not on how we got there. All the information needed for optimal decisions is contained in the current state.
</details>

**3. What's the difference between deterministic and stochastic transitions?**
<details>
<summary>Click to reveal answer</summary>
Deterministic: The outcome of an action is certain (probability 1.0 for one next state).
Stochastic: The outcome is uncertain (probability spread across multiple possible next states).
</details>

**4. In the slippery grid world, if slip probability is 10%, what's the success probability?**
<details>
<summary>Click to reveal answer</summary>
80%. The agent can slip left (10%) or right (10%), so the intended action succeeds with 100% - 10% - 10% = 80%.
</details>

**5. Why is modeling a problem as an MDP useful?**
<details>
<summary>Click to reveal answer</summary>
Once a problem is formulated as an MDP, we can apply standard RL algorithms to find optimal policies. MDPs provide a mathematical framework that makes problems precise and solvable.
</details>

---
## What's Next?

Great job! Now you understand the mathematical framework behind RL.

In the next notebook, we'll dive into **Rewards and Returns** - how to measure the total success of a policy over time. We'll learn about:
- Cumulative rewards
- Discounting and why it matters
- Expected returns

**Continue to:** [Notebook 3: Rewards and Returns](03_rewards_and_returns.ipynb)

---

*You now speak the language of MDPs! Every RL problem you encounter can be described using these 5 components.*